In [ ]:
import os
import asyncio
from deepgram import Deepgram
from dotenv import load_dotenv

In [ ]:
load_dotenv()
DEEPGRAM_API_KEY = os.getenv("DEEPGRAM_API_KEY")

# Set the input folder path
input_folder = "../Audio"

# Set the output folder path
output_folder = "/TT"

# Initialize the Deepgram client
dg_client = Deepgram(DEEPGRAM_API_KEY)

In [ ]:
async def transcribe_audio(audio_file):
    """
    Transcribe a single audio file using the Deepgram API.
    """
    try:
        with open(audio_file, 'rb') as audio:
            response = await dg_client.transcription.prerecorded(audio, {
                'language': 'hi-IN',
                'model': 'nova-2'
            })
        
        # Get the base filename
        base_filename = os.path.splitext(os.path.basename(audio_file))[0]
        
        # Find the next available ID for the output file
        output_id = 1
        output_file = os.path.join(output_folder, f"{base_filename}_{output_id}.txt")
        while os.path.exists(output_file):
            output_id += 1
            output_file = os.path.join(output_folder, f"{base_filename}_{output_id}.txt")
        
        # Save the transcription to a file
        with open(output_file, 'w', encoding='utf-8') as f:
            f.write(response.text)
        
        print(f"Transcription for {audio_file} saved to {output_file}")
    except Exception as e:
        print(f"Error transcribing {audio_file}: {e}")

In [ ]:
async def main():
    """
    Transcribe all audio files in the input folder using asynchronous API calls.
    """
    audio_files = [os.path.join(input_folder, f) for f in os.listdir(input_folder) if f.endswith(".wav")]
    
    await asyncio.gather(*[transcribe_audio(audio_file) for audio_file in audio_files])

if __name__ == "__main__":
    asyncio.run(main())